# 🔐 Blind-Sight: Privacy-Preserving Analytics with Homomorphic Encryption

## Overview
This notebook demonstrates **Blind-Sight** - performing analytics on encrypted data without ever decrypting it during computation. We use **TenSEAL** library with the **CKKS** homomorphic encryption scheme.

### What is Homomorphic Encryption?
Homomorphic Encryption (HE) allows computations on encrypted data:
- **Encrypt** → **Compute** → **Decrypt** = Same result as computing on plaintext
- Data remains private throughout the entire computation
- Perfect for sensitive data analytics (healthcare, finance, interviews)

### What We'll Demonstrate:
1. ✅ Generate sample interview/salary dataset
2. ✅ Encrypt the dataset using CKKS scheme
3. ✅ Perform encrypted analytics (sum, average)
4. ✅ Perform encrypted linear regression
5. ✅ Decrypt and compare with plaintext results

## 1. Install and Import Required Libraries

First, we install TenSEAL (if not available) and import all necessary libraries.

In [ ]:
# ============================================================
# STEP 1: Install TenSEAL Library
# ============================================================
# TenSEAL is a library for homomorphic encryption built on top of Microsoft SEAL
# It provides easy-to-use Python bindings for encrypted computations

# Uncomment the line below if TenSEAL is not installed
# !pip install tenseal

# Install other dependencies if needed
# !pip install numpy pandas matplotlib

In [ ]:
# ============================================================
# STEP 2: Import Required Libraries
# ============================================================

import tenseal as ts          # TenSEAL for homomorphic encryption
import numpy as np            # NumPy for numerical operations
import pandas as pd           # Pandas for data handling
import time                   # For measuring computation time

# Set random seed for reproducibility
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"📦 TenSEAL version: {ts.__version__}")

## 2. Generate Sample Dataset

We'll create a synthetic dataset representing interview candidate data:
- **Salaries**: Annual salary figures (sensitive data)
- **Experience**: Years of work experience
- **Interview Scores**: Performance scores from mock interviews

This simulates real-world sensitive data that needs privacy protection.

In [ ]:
# ============================================================
# STEP 3: Generate Sample Dataset
# ============================================================
# We create synthetic data representing interview candidates
# This data would normally be highly sensitive (salaries, scores)

# Number of candidates in our dataset
NUM_CANDIDATES = 100

# Generate synthetic interview data
# ---------------------------------

# Years of experience (0-20 years)
experience = np.random.uniform(0, 20, NUM_CANDIDATES)

# Interview scores (0-100 scale) - correlated with experience
# More experience generally leads to higher scores (with some noise)
interview_scores = 40 + (experience * 2.5) + np.random.normal(0, 10, NUM_CANDIDATES)
interview_scores = np.clip(interview_scores, 0, 100)  # Keep within 0-100

# Salaries (in thousands) - correlated with experience
# Base salary + experience bonus + random variation
salaries = 30 + (experience * 4) + np.random.normal(0, 8, NUM_CANDIDATES)
salaries = np.clip(salaries, 25, 150)  # Realistic salary range

# Create a pandas DataFrame for nice display
df = pd.DataFrame({
    'Candidate_ID': range(1, NUM_CANDIDATES + 1),
    'Experience_Years': np.round(experience, 2),
    'Interview_Score': np.round(interview_scores, 2),
    'Salary_K': np.round(salaries, 2)
})

print("📊 Sample Dataset Generated!")
print(f"   Total Candidates: {NUM_CANDIDATES}")
print(f"\n🔒 This is SENSITIVE data that needs privacy protection!\n")
print(df.head(10).to_string(index=False))
print("\n... and", NUM_CANDIDATES - 10, "more rows")

## 3. Initialize TenSEAL Context (CKKS Encryption Setup)

The **CKKS scheme** (Cheon-Kim-Kim-Song) is designed for approximate arithmetic on encrypted real numbers. We configure:

- **Polynomial Modulus Degree**: Security parameter (higher = more secure but slower)
- **Coefficient Modulus**: Controls precision and computation depth
- **Global Scale**: Encoding precision for floating-point numbers

```
┌─────────────────────────────────────────────────────────────┐
│                    CKKS ENCRYPTION SCHEME                    │
├─────────────────────────────────────────────────────────────┤
│  Plaintext: [3.14, 2.71, 1.41]                              │
│       ↓                                                      │
│  Encrypt with Public Key                                     │
│       ↓                                                      │
│  Ciphertext: [encrypted polynomial coefficients...]         │
│       ↓                                                      │
│  Homomorphic Operations (+, -, ×)                           │
│       ↓                                                      │
│  Decrypt with Secret Key                                     │
│       ↓                                                      │
│  Result: [approximate result with small error]               │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# STEP 4: Initialize TenSEAL Context for CKKS Encryption
# ============================================================
# The context holds all encryption parameters and keys

def create_tenseal_context():
    """
    Create a TenSEAL context with CKKS scheme parameters.
    
    CKKS allows approximate arithmetic on encrypted floating-point numbers.
    """
    # Create context with CKKS scheme
    context = ts.context(
        scheme=ts.SCHEME_TYPE.CKKS,  # CKKS for real number arithmetic
        
        # Polynomial modulus degree (power of 2)
        # Higher = more security but slower computation
        # 8192 provides ~128-bit security
        poly_modulus_degree=8192,
        
        # Coefficient modulus bit sizes
        # Determines precision and multiplication depth
        # First and last are for encryption/decryption
        # Middle ones are consumed by multiplications
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    
    # Set the global scale for encoding
    # 2^40 provides good precision for most applications
    context.global_scale = 2**40
    
    # Generate Galois keys for rotation operations (needed for sum)
    context.generate_galois_keys()
    
    return context

# Create the encryption context
print("🔑 Creating TenSEAL encryption context...")
context = create_tenseal_context()

print("✅ CKKS Context created successfully!")
print(f"   Polynomial Modulus Degree: 8192")
print(f"   Security Level: ~128-bit")
print(f"   Global Scale: 2^40")
print(f"   Supports: Addition, Subtraction, Multiplication")

## 4. Encrypt the Dataset

Now we encrypt our sensitive data using CKKS encryption. After encryption:
- ❌ Cannot see individual values
- ❌ Cannot identify specific candidates
- ✅ CAN perform mathematical operations
- ✅ CAN compute statistics

**This is the "Blind" part of Blind-Sight!**

In [ ]:
# ============================================================
# STEP 5: Encrypt the Dataset
# ============================================================
# Convert plaintext vectors to encrypted CKKS vectors

print("🔐 Encrypting sensitive data...")
start_time = time.time()

# Encrypt each column of our dataset
# ts.ckks_vector creates an encrypted vector from plaintext

# Encrypt experience years
encrypted_experience = ts.ckks_vector(context, experience.tolist())

# Encrypt interview scores
encrypted_scores = ts.ckks_vector(context, interview_scores.tolist())

# Encrypt salaries (most sensitive!)
encrypted_salaries = ts.ckks_vector(context, salaries.tolist())

encryption_time = time.time() - start_time

print(f"✅ Dataset encrypted successfully!")
print(f"   ⏱️  Encryption time: {encryption_time:.3f} seconds")
print(f"\n📊 Encrypted Vectors:")
print(f"   • Experience: {len(experience)} values encrypted")
print(f"   • Scores:     {len(interview_scores)} values encrypted")
print(f"   • Salaries:   {len(salaries)} values encrypted")

# Show that encrypted data is NOT readable
print(f"\n🔒 Encrypted data preview (first 50 chars of serialized):")
print(f"   {encrypted_salaries.serialize()[:50]}...")
print(f"\n   ❌ Individual values are now HIDDEN!")

## 5. Perform Encrypted Analytics - SUM Operation

Now the magic happens! We compute the **SUM** of all salaries while they remain encrypted.

```
┌─────────────────────────────────────────────────────┐
│           HOMOMORPHIC SUM OPERATION                  │
├─────────────────────────────────────────────────────┤
│  Encrypted: [🔒, 🔒, 🔒, 🔒, 🔒]                     │
│       ↓                                              │
│  Sum Operation (on encrypted data)                   │
│       ↓                                              │
│  Encrypted Result: 🔒                                │
│       ↓                                              │
│  Decrypt → Actual Sum!                               │
└─────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# STEP 6: Perform Encrypted SUM Operation
# ============================================================
# Sum all values WITHOUT decrypting individual entries

print("📊 Computing SUM on encrypted data...")
start_time = time.time()

# Method: Use TenSEAL's sum() function on encrypted vector
# This sums all encrypted elements using homomorphic addition

# Sum of encrypted salaries
encrypted_salary_sum = encrypted_salaries.sum()

# Sum of encrypted scores
encrypted_score_sum = encrypted_scores.sum()

# Sum of encrypted experience
encrypted_exp_sum = encrypted_experience.sum()

sum_time = time.time() - start_time

print(f"✅ Encrypted SUM computed!")
print(f"   ⏱️  Computation time: {sum_time:.4f} seconds")
print(f"\n   📦 Results are STILL ENCRYPTED:")
print(f"   • Salary sum: [encrypted - cannot view yet]")
print(f"   • Score sum:  [encrypted - cannot view yet]")
print(f"   • Exp sum:    [encrypted - cannot view yet]")

## 6. Perform Encrypted Analytics - AVERAGE Operation

Calculate the **AVERAGE** on encrypted data by:
1. Computing the encrypted sum (already done)
2. Dividing by count (scalar multiplication with 1/n)

In [ ]:
# ============================================================
# STEP 6: ENCRYPTED AVERAGE CALCULATION
# ============================================================
# Average = Sum / Count
# In CKKS, we multiply by (1/count) since division isn't directly supported

print("=" * 60)
print("ENCRYPTED AVERAGE OPERATION")
print("=" * 60)

n_samples = len(df)
scale_factor = 1.0 / n_samples

# Calculate encrypted averages by multiplying sum by (1/n)
start_time = time.time()

# Method 1: Using the sum we already computed
avg_salary_encrypted = encrypted_sum_salary * scale_factor
avg_score_encrypted = encrypted_sum_score * scale_factor
avg_experience_encrypted = encrypted_sum_experience * scale_factor

avg_time = time.time() - start_time

print(f"\n✅ Encrypted averages calculated in {avg_time:.6f} seconds")
print(f"   Scale factor used: 1/{n_samples} = {scale_factor:.6f}")
print(f"\n📊 Encrypted average vectors created:")
print(f"   - avg_salary_encrypted")
print(f"   - avg_score_encrypted")
print(f"   - avg_experience_encrypted")

## 7. Perform Encrypted Analytics - LINEAR REGRESSION

Implement **simple linear regression** on encrypted data:
- Predict `expected_salary` from `interview_score`
- Formula: $\hat{y} = \beta_0 + \beta_1 x$

Where:
- $\beta_1 = \frac{\sum(x_i - \bar{x})(y_i - \bar{y})}{\sum(x_i - \bar{x})^2}$
- $\beta_0 = \bar{y} - \beta_1 \bar{x}$

**Note:** Due to CKKS limitations on multiplicative depth, we'll demonstrate encrypted weighted sum which is the core operation for linear predictions.

In [ ]:
# ============================================================
# STEP 7: ENCRYPTED LINEAR REGRESSION COMPONENTS
# ============================================================
# We'll compute the core components needed for linear regression
# on encrypted data: dot products and weighted sums

print("=" * 60)
print("ENCRYPTED LINEAR REGRESSION COMPONENTS")
print("=" * 60)

start_time = time.time()

# For linear regression: predict salary from score
# We need: sum(x*y), sum(x^2), sum(x), sum(y)

# Encrypted dot product: sum(score * salary)
# Element-wise multiplication of encrypted vectors
encrypted_xy = encrypted_scores * encrypted_salaries  # Element-wise product
encrypted_dot_product = encrypted_xy.sum()

# Encrypted sum of squares: sum(score^2)
encrypted_x_squared = encrypted_scores * encrypted_scores
encrypted_sum_x_squared = encrypted_x_squared.sum()

# We already have sum(x) = encrypted_sum_score and sum(y) = encrypted_sum_salary

regression_time = time.time() - start_time

print(f"\n✅ Encrypted regression components computed in {regression_time:.6f} seconds")
print(f"\n📊 Components calculated (all encrypted):")
print(f"   - Σ(xy): encrypted_dot_product")
print(f"   - Σ(x²): encrypted_sum_x_squared")
print(f"   - Σx: encrypted_sum_score (from Step 5)")
print(f"   - Σy: encrypted_sum_salary (from Step 5)")
print(f"\n🔒 All operations performed on encrypted data!")
print(f"   No individual scores or salaries were exposed.")

## 8. Decrypt Results

Now we decrypt the encrypted analytics results using our CKKS context.
The decryption reveals the **aggregate statistics** only - individual data points remain protected.

In [ ]:
# ============================================================
# STEP 8: DECRYPT ALL RESULTS
# ============================================================

print("=" * 60)
print("DECRYPTING ENCRYPTED ANALYTICS RESULTS")
print("=" * 60)

start_time = time.time()

# Decrypt SUM results
decrypted_sum_salary = encrypted_sum_salary.decrypt()[0]
decrypted_sum_score = encrypted_sum_score.decrypt()[0]
decrypted_sum_experience = encrypted_sum_experience.decrypt()[0]

# Decrypt AVERAGE results
decrypted_avg_salary = avg_salary_encrypted.decrypt()[0]
decrypted_avg_score = avg_score_encrypted.decrypt()[0]
decrypted_avg_experience = avg_experience_encrypted.decrypt()[0]

# Decrypt LINEAR REGRESSION components
decrypted_dot_product = encrypted_dot_product.decrypt()[0]
decrypted_sum_x_squared = encrypted_sum_x_squared.decrypt()[0]

decrypt_time = time.time() - start_time

print(f"\n✅ Decryption completed in {decrypt_time:.6f} seconds")

print(f"\n{'='*60}")
print("DECRYPTED RESULTS:")
print(f"{'='*60}")

print(f"\n📈 SUM Results:")
print(f"   Sum of Salaries:    ${decrypted_sum_salary:,.2f}")
print(f"   Sum of Scores:      {decrypted_sum_score:,.2f}")
print(f"   Sum of Experience:  {decrypted_sum_experience:,.2f} years")

print(f"\n📊 AVERAGE Results:")
print(f"   Avg Salary:         ${decrypted_avg_salary:,.2f}")
print(f"   Avg Score:          {decrypted_avg_score:.2f}")
print(f"   Avg Experience:     {decrypted_avg_experience:.2f} years")

print(f"\n📉 LINEAR REGRESSION Components:")
print(f"   Σ(xy):              {decrypted_dot_product:,.2f}")
print(f"   Σ(x²):              {decrypted_sum_x_squared:,.2f}")

## 9. Compare with Plaintext Computation

Compute the same analytics on **unencrypted (plaintext)** data using NumPy/Pandas.
This allows us to verify the accuracy of our homomorphic encryption operations.

In [ ]:
# ============================================================
# STEP 9: PLAINTEXT (UNENCRYPTED) COMPUTATION
# ============================================================

print("=" * 60)
print("PLAINTEXT COMPUTATION FOR COMPARISON")
print("=" * 60)

# Compute the same statistics on plaintext data
plaintext_sum_salary = np.sum(salaries)
plaintext_sum_score = np.sum(scores)
plaintext_sum_experience = np.sum(experience)

plaintext_avg_salary = np.mean(salaries)
plaintext_avg_score = np.mean(scores)
plaintext_avg_experience = np.mean(experience)

# Linear regression components
plaintext_dot_product = np.sum(scores * salaries)
plaintext_sum_x_squared = np.sum(scores * scores)

# Calculate actual linear regression coefficients
n = len(scores)
beta_1 = (n * plaintext_dot_product - plaintext_sum_score * plaintext_sum_salary) / \
         (n * plaintext_sum_x_squared - plaintext_sum_score ** 2)
beta_0 = plaintext_avg_salary - beta_1 * plaintext_avg_score

print(f"\n📈 PLAINTEXT SUM Results:")
print(f"   Sum of Salaries:    ${plaintext_sum_salary:,.2f}")
print(f"   Sum of Scores:      {plaintext_sum_score:,.2f}")
print(f"   Sum of Experience:  {plaintext_sum_experience:,.2f} years")

print(f"\n📊 PLAINTEXT AVERAGE Results:")
print(f"   Avg Salary:         ${plaintext_avg_salary:,.2f}")
print(f"   Avg Score:          {plaintext_avg_score:.2f}")
print(f"   Avg Experience:     {plaintext_avg_experience:.2f} years")

print(f"\n📉 PLAINTEXT LINEAR REGRESSION Components:")
print(f"   Σ(xy):              {plaintext_dot_product:,.2f}")
print(f"   Σ(x²):              {plaintext_sum_x_squared:,.2f}")

print(f"\n🔮 REGRESSION COEFFICIENTS (from plaintext):")
print(f"   β₀ (intercept):     ${beta_0:,.2f}")
print(f"   β₁ (slope):         ${beta_1:,.2f}")
print(f"\n   Model: Salary = ${beta_0:,.2f} + ${beta_1:,.2f} × Score")

## 10. Verify Accuracy & Final Comparison

Compare encrypted vs plaintext results to verify the accuracy of CKKS homomorphic encryption.
CKKS is an **approximate** scheme, so we expect small numerical differences (typically < 0.001% for well-configured parameters).

In [ ]:
# ============================================================
# STEP 10: ACCURACY VERIFICATION & COMPARISON
# ============================================================

print("=" * 60)
print("ACCURACY VERIFICATION: ENCRYPTED vs PLAINTEXT")
print("=" * 60)

def calculate_error(encrypted_val, plaintext_val):
    """Calculate absolute and relative error."""
    abs_error = abs(encrypted_val - plaintext_val)
    rel_error = (abs_error / abs(plaintext_val)) * 100 if plaintext_val != 0 else 0
    return abs_error, rel_error

# Calculate errors for all metrics
errors = {
    "Sum Salary": calculate_error(decrypted_sum_salary, plaintext_sum_salary),
    "Sum Score": calculate_error(decrypted_sum_score, plaintext_sum_score),
    "Sum Experience": calculate_error(decrypted_sum_experience, plaintext_sum_experience),
    "Avg Salary": calculate_error(decrypted_avg_salary, plaintext_avg_salary),
    "Avg Score": calculate_error(decrypted_avg_score, plaintext_avg_score),
    "Avg Experience": calculate_error(decrypted_avg_experience, plaintext_avg_experience),
    "Σ(xy)": calculate_error(decrypted_dot_product, plaintext_dot_product),
    "Σ(x²)": calculate_error(decrypted_sum_x_squared, plaintext_sum_x_squared),
}

# Display comparison table
print(f"\n{'Metric':<20} {'Encrypted':<18} {'Plaintext':<18} {'Abs Error':<15} {'Rel Error':<12}")
print("-" * 85)

comparison_data = [
    ("Sum Salary", decrypted_sum_salary, plaintext_sum_salary),
    ("Sum Score", decrypted_sum_score, plaintext_sum_score),
    ("Sum Experience", decrypted_sum_experience, plaintext_sum_experience),
    ("Avg Salary", decrypted_avg_salary, plaintext_avg_salary),
    ("Avg Score", decrypted_avg_score, plaintext_avg_score),
    ("Avg Experience", decrypted_avg_experience, plaintext_avg_experience),
    ("Σ(xy)", decrypted_dot_product, plaintext_dot_product),
    ("Σ(x²)", decrypted_sum_x_squared, plaintext_sum_x_squared),
]

all_passed = True
for metric, enc_val, plain_val in comparison_data:
    abs_err, rel_err = errors[metric]
    status = "✅" if rel_err < 0.01 else "⚠️"  # < 0.01% is excellent
    if rel_err >= 0.01:
        all_passed = False
    print(f"{metric:<20} {enc_val:<18,.4f} {plain_val:<18,.4f} {abs_err:<15,.6f} {rel_err:<10,.8f}%")

print("-" * 85)

# Summary
print(f"\n{'='*60}")
print("VERIFICATION SUMMARY")
print(f"{'='*60}")

max_rel_error = max(err[1] for err in errors.values())
avg_rel_error = sum(err[1] for err in errors.values()) / len(errors)

print(f"\n📊 Error Statistics:")
print(f"   Maximum Relative Error: {max_rel_error:.10f}%")
print(f"   Average Relative Error: {avg_rel_error:.10f}%")

if max_rel_error < 0.01:
    print(f"\n✅ SUCCESS: All computations within acceptable accuracy!")
    print(f"   CKKS homomorphic encryption is working correctly.")
elif max_rel_error < 1.0:
    print(f"\n⚠️ WARNING: Some variance detected, but within reasonable bounds.")
else:
    print(f"\n❌ ERROR: Significant accuracy issues detected.")

## 11. Conclusion & Key Takeaways

### 🎯 What We Demonstrated

**Blind-Sight Analytics** using Homomorphic Encryption enables:

1. **Privacy-Preserving Computation** - Analytics performed on encrypted data without exposing individual records
2. **Secure Aggregation** - Sum, average, and regression components calculated without decryption
3. **Verifiable Accuracy** - Results match plaintext computation within CKKS approximation bounds

### 🔐 Security Properties

| Property | Description |
|----------|-------------|
| **Data Confidentiality** | Individual values never exposed during computation |
| **Aggregate Analytics** | Only statistical summaries are revealed |
| **No Trusted Third Party** | Computations can be outsourced to untrusted servers |

### 📚 Use Cases for AI Mock Interview System

- **Salary Benchmarking** - Compute market averages without exposing individual offers
- **Performance Analytics** - Aggregate interview scores across companies privately
- **Bias Detection** - Statistical analysis without accessing protected attributes

### ⚠️ Limitations

- **Computational Overhead** - HE operations are slower than plaintext
- **Multiplicative Depth** - Limited number of multiplications before noise grows too large
- **Approximate Results** - CKKS provides approximate (not exact) arithmetic